# P2: Plotting and interpreting data

Contents 

0. [Introduction](##introduction)
1. .
2. .
3. [Plots](###simple-ppi-plots)
4. .
5. 

## 1. Introduction 

In this notebook we will make use of [The Python ARM Radar Toolkit (Py-ART)](https://arm-doe.github.io/pyart/), a Python module containing a collection of tools to processes and explore data from many types of weather radars. Py-ART can read data in common weather radar formats, produce plots (e.g. PPIs and RHIs) and apply common data corrections. In this practical, we will use Py-ART to read data from the NXPol radar, plot the data and customise the plots. You will then be able to use this to explore different data and identify meteorological features. 

First let's import Py-ART and other packages needed to run the code. For this purposes of this course, these have been installed for you but when using your own computer you will need to install Py-ART and the required dependencies. Futher details can be found [here](https://arm-doe.github.io/pyart/#install). 

In [ ]:
import pyart 
import numpy as np # for working with data arrays 
import matplotlib.pyplot as plt # to make and customise plots
import cartopy.crs as ccrs # to produce maps
import glob # for accessing multiple files using wild cards (useful for finding and reading data files)
import os # for operating system dependent functionality such as manipulating file paths (useful for finding and reading data files)

## 2. Reading data

As outline in Practical 1, Py-ART can read data in common weather radar formats (e.g. Sigmet/IRIS, MDV, CF/Radial, UF, NEXRAD). We will start with an example radar file from NXPol-2.

In [ ]:
radar_file = '/gws/pw/j07/woest/data/ncas-mobile-x-band-radar-1//level2/sur/20230622/bl_scans/ncas-mobile-x-band-radar-1_lyneham_20230622-182423_vol_v1.0.0.nc'

When reading data we will use the [```pyart.io.read```](https://arm-doe.github.io/pyart/API/generated/pyart.io.read.html) module. ```pyart.io.read``` can automatically detect the file format, however in some cases explicitely choosing the appropriate function for the file format may be necessary. Documentation on what formats Py-ART can read can be found [here](https://arm-doe.github.io/pyart/API/generated/pyart.io.html). NXPol data is in the common CF-radial format so we can simply use ```pyart.io.read```. Setting ```delay_field_loading=True``` delays loading of data from the file until it is accessed, this can speed up loading. 

In [ ]:
radar = pyart.io.read(radar_file, delay_field_loading=True)

When we run this, we get a [```pyart.Radar``` object](https://arm-doe.github.io/pyart/API/generated/pyart.core.Radar.html#pyart.core.Radar), within which the variables are stored in dictionaries or dictionaries of dictionaries. (Note: for more information on Python dictionaries see [here](https://docs.python.org/3/tutorial/datastructures.html#dictionaries)). Let's have a look using the ```radar.info``` method. 

As you can see, this gives a complete picture of what is in the file. Information about the radar, data processing and observational period can be found in the ```radar.metadata``` attribute. Variables relating to the scans are stored in dictionaries that contain the data as well important information such as the units and variable name (according to C/F conventions). We can access these using the radar object and the variable name. For example, we can look at the elevation. 

In [ ]:
radar.elevation

The ```fields``` attribute stores the actual measurement data such as reflectivity and velocity in a dictionary of dictionaries. To see what fields are present let's access the dictionary keys.

In [ ]:
radar.fields.keys()

To look at an individual field we can use the variable name to access the relevant dictionary. For example, to look at the horizontal reflectivity we use ```'dBZ'```. Note that because we set ```delay_field_loading=True``` when reading the data the field attribute contains LazyLoadDict objects no dict objects and the field data is not loaded until the 'data' key is accessed. To have a look at the dictionary we need to print the field and not just call ```radar.fields['dBZ']``` as we could with a normal dict object.

In [ ]:
print(radar.fields['dBZ'])

Within each field the data itself is stored under the ```'data'``` key which can be added to the call to extract the data array from the dictionary.

In [ ]:
reflectivity = radar.fields['dBZ']['data']
reflectivity

## 3. Plotting data

### 3.1 Simple PPI plots
Having read in the data, we can now make some plots. Py-ART has built in visualisation classes for plotting PPIs, RHIs and individual ray traces in the [```pyart.graph``` module](https://arm-doe.github.io/pyart/API/generated/pyart.graph.html). 

The first step to producing a plot using Py-ART is to initilize a [radar display object](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.html) using our radar object as the input to the function. 

In [ ]:
display = pyart.graph.RadarDisplay(radar)

Once the the map display is initialized we can add a plot to it Py-ART's plotting functions. The types of plots available can be found in the [```RadarDisplay``` documentation](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.html#pyart.graph.RadarDisplay). The data we have loaded is from NXPol used in PPI mode, meaning the elevation angle is constant but the azimuth angle varies (i.e. the radar rotates through 360 degrees). We can display this data on a 2D PPI plot with the radar in the centre. 

In [ ]:
display.plot_ppi('dBZ')

This plot is at 1.5 deg elevation angle but having a look at the ```radar.nsweeps``` attribute reveals that there are are 6 sweeps, each at a different elevation angle. We can specify the sweep index when making the plot to choose a different elevation angle. Note: try ```np.unique(radar.elevation['data'])``` to see what elevation angles there are.

In [ ]:
display.plot_ppi('dBZ', 2) 

As well as plotting the PPI we can add other useful details to the map such as range rings and crosshairs to show the location of the radar. Each plotting function also takes various input variables, outlined in the documentation (e.g. [```plot_ppi``` documentation](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_ppi.html#pyart.graph.RadarDisplay.plot_ppi)), that we can play with to being to customise the plot. Let's see what we can do. 

In [ ]:
display.plot_ppi('dBZ', 
                 sweep = 0, # sweep index
                 vmin = -10, # lower limit of colourscale 
                 vmax = 50, # upper limit of colourscale 
                 title = 'This is a PPI plot', # set title (can also be turned off using title_flag = False)
                 colorbar_label  = 'Horizontal reflectivity (dBZ)', # set colourbar label
                 cmap = 'viridis' # change colourmap (we will come back to colourmaps later...)
                )

display.plot_range_rings([50,100], # list of locations to draw range rings in km from radar
                         ls = '-', #linestyle
                         lw = '1' #linewidth
                         ) 

display.plot_cross_hair(2) # size in km

display.plot_grid_lines(col = 'gray', # colour
                        ls = ':' # linestyle 
                        ) 

### 3.2 Creating a map
Our PPI plot is a good start but it is usually beneficial to know where our radar is located. Luckily for us, Py-ART can also create plots on a geographic map using [Cartopy](https://scitools.org.uk/cartopy/docs/latest/). The process is similar to what we have just done, but rather than a radar display object we inizialize a [radar map display object](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.html) upon which we can plot a PPI map.

In [ ]:
display = pyart.graph.RadarMapDisplay(radar)
display.plot_ppi_map('dBZ', 0)

As you can see, we now have some new features (but what are they?) and the X, Y labels in distance from the radar are no longer present. You may also have noticed an error informing you that no projection has been defined for the axes. When creating a map, the projection defines the set of transformations used to flatten the curved surface of the globe on to a plane. It is impossible to do this without some sort of distortion, and different map projections distort the globe in different ways. Some projections maintain the shape of objects but distort their area, leading to some places appearing much larger than they are. Others maintain the area but distort their shape, leading to places appearing squashed or stretched. The 'best' projection to use varies depending on the desired use, location and data being plotted. For radar data, the azimuthal equidistant projection centred on the radar is a good choice because this projection preserves distance and direction from the central point. 

We can get the radar longitude and latitude from the radar object and use these to define the Cartopy projection to be used. 

In [ ]:
grid_projection = ccrs.AzimuthalEquidistant(central_longitude=radar.longitude['data'][0],
                                            central_latitude=radar.latitude['data'][0]
                                           )


Checking out the [```plot_ppi_map``` documentation](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarMapDisplay.plot_ppi_map.html) shows how we can set the projection when plotting the map and gives additional ways we can customise the plots, for example changing the map resolution.

In [ ]:
display.plot_ppi_map('dBZ', # field 
                     sweep = 0, # sweep index
                     resolution = '10m', # map resolution
                     vmin = -10, # lower limit of colourscale 
                     vmax = 50, # upper limit of colourscale 
                     title = 'This is a PPI map plot', # set title (can also be turned off using title_flag = False)
                     colorbar_label  = 'Horizontal reflectivity (dBZ)', # set colourbar label
                     projection = grid_projection
                    )

display.plot_range_rings([50,100], 
                         ls = '--',
                         lw = '1')

display.plot_cross_hair(2000)

### 3.3 Customising plots 

Py-ART plotting functions are great however the customisations can sometimes be limited, and you may wish to personalise things more. Since Py-ART plotting is built upon matplotlib and cartopy we can use these as we would with any other type of plotting by turning off the embellishments added by Py-ART and adding our own. We can also add other features to the map, for example plotting the location of Chilbolton Atmospheric Observatory. 

In [ ]:
sweep_idx = 0

fig = plt.figure(figsize=(8,5)) # Make the figure bigger
ax = plt.subplot(111, projection=grid_projection) # create GeoAxes using the correct projection

# plot ppi
display.plot_ppi_map('dBZ',
                     sweep = 0,
                     ax = ax,
                     embellish = False, # turn off embellishments such as coastlines
                     title_flag = False, # turn off title
                     projection = grid_projection
                     )

# add coastlines to plot (this adds coastlines only, without the admin boundaries that Py-ART includes)
ax.coastlines(resolution = '10m');

# set up title and add 
start_dt = radar.metadata['start_datetime']
elevation = np.unique(radar.elevation['data'])[sweep_idx]
instrument = 'NXPol-1'
title = f'{instrument}, {start_dt}, elevation {elevation}$^\\circ$'
ax.set_title(title, fontsize = 12)

# plot range ringes
display.plot_range_rings([50,100], 
                         ls = '--',
                         lw = '1')

# plot cross hair
display.plot_cross_hair(2000, ax = ax) 

# add gridlines 
gl = ax.gridlines(draw_labels=True,
                  dms = False,
                  x_inline=False,
                  y_inline=False,
                  rotate_labels=False,
                  alpha = 0.7,
                  zorder = 0,
                  linewidth = 0.25,
                  )
gl.right_labels = gl.top_labels = False

# plot Chilbolton location
chilbolton_lon, chilbolton_lat = -1.43858, 51.14456

ax.plot(chilbolton_lon, chilbolton_lat, 
        transform = ccrs.PlateCarree(), 
        linestyle = '',
        marker = 'D',
        markersize = 4,
        color = 'black',
        label = 'Chilbolton atmospheric observatory'
       )

# add legend
ax.legend(loc = 'upper right',fontsize = 9);


### 3.4 Plotting multiple variables on subplots 

Info here

In [ ]:
fig = plt.figure(figsize=(11,4), constrained_layout = True) # Make the figure bigger
ax1 = plt.subplot(121, projection=grid_projection)
ax2 = plt.subplot(122, projection=grid_projection)

# plot ppi

plot_variables = ['dBZ','V']
axes = [ax1,ax2]
for ax,field in zip(axes,plot_variables):
    display.plot_ppi_map(field,
                     sweep = 0,
                     ax = ax, # set which axes to create the plot on 
                     embellish = False, # turn off embellishments such as coastlines
                     title_flag = False, # turn off title
                     projection = grid_projection # set projection
                     )
    
    # add coastlines to plot
    ax.coastlines(resolution = '10m');
    
    # set up title and add 
    start_dt = radar.metadata['start_datetime']
    elevation = np.unique(radar.elevation['data'])[sweep_idx]
    instrument = 'NXPol-1'
    title = f'{instrument}, {start_dt}, elevation {elevation}$^\\circ$'
    #ax.set_title(title, fontsize = 12)
    
    # plot range ringes
    display.plot_range_rings([50,100],
                             ls = '--',
                             lw = '1')
    
    # plot cross hair
    display.plot_cross_hair(2000, ax = ax) 
    
    # add gridlines 
    gl = ax.gridlines(draw_labels=True,
                      dms = False,
                      x_inline=False,
                      y_inline=False,
                      rotate_labels=False,
                      alpha = 0.7,
                      zorder = 0,
                      linewidth = 0.25,
                      )
    gl.right_labels = gl.top_labels = False
    
    # plot Chilbolton location
    chilbolton_lon, chilbolton_lat = -1.43858, 51.14456
    
    ax.plot(chilbolton_lon, chilbolton_lat, 
            transform = ccrs.PlateCarree(), 
            linestyle = '',
            marker = 'D',
            markersize = 4,
            color = 'black',
            label = 'Chilbolton atmospheric observatory'
           )
    
    # add legend
    ax.legend(loc = 'upper right',fontsize = 9);

fig.suptitle(title);

## Create RHI plots 

As well as PPI scans, NXPol can also run in Range Height Indicator (RHI) mode, meaning the azimuth angle is fixed but the elevation changes leading to data in the vertical plane. We can make RHI plots using Py-ART by once again initilizing a radar display and then using the [```display.plot_rhi```](https://arm-doe.github.io/pyart/API/generated/pyart.graph.RadarDisplay.plot_rhi.html) function. 

In [ ]:
radar_file = '/gws/pw/j07/woest/data/ncas-mobile-x-band-radar-1/level2/rhi/20230612/ncas-mobile-x-band-radar-1_lyneham_20230612-151859_rhi_v1.0.0.nc'

In [ ]:
radar = pyart.io.read(radar_file, delay_field_loading=True)

In [ ]:
display = pyart.graph.RadarDisplay(radar)
display.plot_rhi('dBZ')

This can be customised in a similar way to our previous plots. 

In [ ]:
fig = plt.figure(figsize=(12,4)) # Make the figure bigger
ax = plt.subplot(111)
                 
display.plot_rhi('dBZ',
                 ax = ax,
                 vmax = 50,
                 zorder = 10)

ax.set_ylim([0,10])
ax.set_xlim([0,100])
ax.grid(alpha = 0.5,
        zorder = 0)

In [ ]:
fig = plt.figure(figsize=(11,6), constrained_layout = True) # Make the figure bigger
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)

# plot ppi

plot_variables = ['dBZ','V']
axes = [ax1,ax2]
for ax,field in zip(axes,plot_variables):
    display.plot_rhi(field,
                     ax = ax,
                     title_flag = False, # turn off title
                     )
    
    
    # set up title and add 
    start_dt = radar.metadata['start_datetime']
    elevation = np.unique(radar.elevation['data'])[sweep_idx]
    instrument = 'NXPol-1'
    title = f'{instrument}, {start_dt}, elevation {elevation}$^\\circ$'
    #ax.set_title(title, fontsize = 12)

    ax.set_ylim([0,10])
    ax.set_xlim([0,100])
    ax.grid(alpha = 0.5,
            zorder = 0)
    ax.label_outer()


## Choosing a colormap


## Saving plots 
incl. naming conventions 

# P3: Plotting dual-polarisation data

## Create multiple plots
choose different variables in the same file, read in multiple files, loop over subplots, adjust names and filenames in loop

## Brief overview of other ways to plot data

xarray etc

## Notes to self
(will be deleted) 

In [ ]:
#radar_file = '/gws/pw/j07/woest/data/ncas-radar-x-band-2/level2/sur/20230622/bl_scans/ncas-radar-x-band-2_cao_20230622-181414_vol_v1.0.0.nc'

#directory = '/gws/pw/j07/woest/data/ncas-mobile-x-band-radar-1//level2/sur/20230622/bl_scans/'
#files = os.listdir(directory)
#radar_file = os.path.join(directory, files[110])


#directory = '/gws/pw/j07/woest/data/ncas-mobile-x-band-radar-1/level2/rhi/20230612'
#files = os.listdir(directory)
#radar_file = os.path.join(directory, files[85])